# Transfer Learning using CIFAR-10 dataset

#### Name Origin: The Canadian Institute for Advanced Research (CIFAR) is a Canadian-based global research organization that brings together teams of top researchers from around the world to address important and complex questions. 
The CIFAR-10 dataset is a collection of images that are commonly used to train machine learning and computer vision algorithms. It is one of the most widely used datasets for machine learning research. The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images. 

In [1]:
import numpy as np
import pandas as pd
import random
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer 
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import time

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K

In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0





# Convert labels to one-hot encoding
num_classes = 10  # CIFAR-10 has 10 classes
y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)







# Print the shapes of the loaded data and one-hot encoded labels
print(f"x_train shape: {x_train.shape}, y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape}, y_test shape: {y_test.shape}")


x_train shape: (50000, 32, 32, 3), y_train shape: (50000, 10)
x_test shape: (10000, 32, 32, 3), y_test shape: (10000, 10)


In [2]:
import tensorflow as tf

class SimpleMLP:
    @staticmethod
    def build():
        # Load the pre-trained VGG19 model
        base_model = tf.keras.applications.VGG19(include_top=False, weights='imagenet', input_shape=(32, 32, 3))
        # Freeze all layers except the last two convolutional layers and the classification layer
#         for layer in base_model.layers[:-5]:
#             layer.trainable = False
        base_model.trainable=False
        # Create the transfer learning model by adding custom classification layers on top of the base model
        model2 = tf.keras.models.Sequential([
            base_model,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dense(10, activation='softmax')  # Adjust the number of output classes accordingly
        ])
'''Function: It calculates the average value for each feature map across the spatial dimensions (height and width), effectively reducing each feature map
to a single average value.
Input Shape: The expected input is a 4D tensor with shape (batch_size, height, width, channels) if data_format='channels_last', or (batch_size, channels, height, width) if data_format='channels_first'.
Output Shape: The output is a 2D tensor with shape (batch_size, channels). If the keepdims argument is set to True, the output will retain the reduced 
spatial dimensions with length 1, resulting in a 4D tensor.
Usage: This layer is often used in CNNs before the final classification layer to reduce the spatial dimensions of the input and to minimize overfitting
by focusing on the most important features.'''

        # Create an optimizer with the specified learning rate
        custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

        # Compile the model with the custom optimizer
        model2.compile(optimizer=custom_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

        return model2
    
# Create an instance of the SimpleMLP model
simple_mlp_model = SimpleMLP.build()
global_model=simple_mlp_model

# Display the model summary
simple_mlp_model.summary()

80134624/80134624 [==============================] - 187s 2us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 1, 1, 512)         20024384  
                                                                 
 global_average_pooling2d (  (None, 512)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 512)               262656    
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 20292170 (77.41 MB)
Trainable params: 267786 (1.02 MB)
Non-trainable params: 20024384 (76.39 MB)
_________________________________________________________________


In [5]:
import tensorflow as tf
from keras.utils import to_categorical
import numpy as np

# Number of clients
num_clients = 4

# Initialize data structures for each client
client_data = {f'client{i + 1}': {'images': [], 'labels': []} for i in range(num_clients)}

# Define the distribution of classes among clients (modify as needed)
class_distribution = [0.1, 0.2, 0.3, 0.4]  # Adjust the percentages based on your requirements

# Ensure the distribution sums to 1
class_distribution = np.array(class_distribution) / sum(class_distribution)

# Split the data into clients based on the defined distribution
for label in np.unique(y_train):
    label_indices = np.where(y_train == label)[0]
    
    # Shuffle indices for randomness
    np.random.shuffle(label_indices)

    # Distribute indices to clients based on the specified distribution
    cumulative_distribution = np.cumsum(class_distribution)
    for i in range(num_clients):
        start_index = int(cumulative_distribution[i - 1] * len(label_indices)) if i > 0 else 0
        end_index = int(cumulative_distribution[i] * len(label_indices))
        client_indices = label_indices[start_index:end_index]

        # Assign images and labels to the client
        client_data[f'client{i + 1}']['images'].extend(x_train[client_indices])
        client_data[f'client{i + 1}']['labels'].extend(y_train[client_indices])

# Convert lists to NumPy arrays
for client_id in client_data.keys():
    client_data[client_id]['images'] = np.array(client_data[client_id]['images'])
    client_data[client_id]['labels'] = np.array(client_data[client_id]['labels'])

# # One-hot encode labels
# client_data[client_id]['one_hot_labels'] = tf.keras.utils.to_categorical(client_data[client_id]['labels'], num_classes=10)

# Display the number of images for each client
for client_id, data in client_data.items():
    print(f"{client_id}: {len(data['images'])} images")


# Assuming you have client_data and each client's labels
label1 = client_data['client1']['labels']
label2 = client_data['client2']['labels']
label3 = client_data['client3']['labels']
label4 = client_data['client4']['labels']

# Convert labels to one-hot encoding
num_classes = 10 
one_hot_label1 = to_categorical(label1, num_classes=num_classes)
one_hot_label2 = to_categorical(label2, num_classes=num_classes)
one_hot_label3 = to_categorical(label3, num_classes=num_classes)
one_hot_label4 = to_categorical(label4, num_classes=num_classes)

# Now, 'one_hot_labels' can be added to the client_data dictionary

client_data['client1']['one_hot_labels'] = one_hot_label1
client_data['client2']['one_hot_labels'] = one_hot_label2
client_data['client3']['one_hot_labels'] = one_hot_label3
client_data['client4']['one_hot_labels'] = one_hot_label4
   
train1 = client_data['client1']['images']
train2 = client_data['client2']['images']
train3 = client_data['client3']['images']
train4 = client_data['client4']['images']


client1: 50000 images
client2: 100000 images
client3: 150000 images
client4: 200000 images


In [6]:
def create_clients(data_dict):
    '''
    Return a dictionary with keys as client names and values as data and label lists.
    Args: data_dict: A dictionary where keys are client names, and values are tuples of data and labels.
                    For example, {'client_1': (data_1, labels_1), 'client_2': (data_2, labels_2), ...}
    Returns: A dictionary with keys as client names and values as tuples of data and label lists.
    '''
    return data_dict

import tensorflow as tf


def test_model(x_test, y_test,  model, comm_round):
    loss,accuracy=model.evaluate(x_test, y_test)
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, accuracy, loss))
    return accuracy, loss


def avg_weights(scaled_weight_list):
    '''Return the average of the listed scaled weights.'''
    num_clients = len(scaled_weight_list)

    if num_clients == 0:
        return None  # Handle the case where the list is empty

    avg_grad = list()

    # Get the sum of gradients across all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0) / num_clients
        avg_grad.append(layer_mean)

    return avg_grad


client_data = {
    'client_1': (train1,label1),
    'client_2': (train2,label2),
    'client_3': (train3,label3),
    'client_4': (train4,label4)
    
}

#create clients
clients_batched = create_clients(client_data)

In [7]:
client_names = list(clients_batched.keys())

In [10]:
# from keras.utils import Sequence
# import numpy as np
# from tqdm import tqdm
# import random
# from keras import backend as K

comms_round = 30  # Number of global epochs
acc3 = []

# class DataGenerator(Sequence):
#     def __init__(self, data, labels, batch_size):
#         self.data = data
#         self.labels = labels
#         self.batch_size = batch_size

#     def __len__(self):
#         return int(np.ceil(len(self.data) / self.batch_size))

#     def __getitem__(self, index):
#         batch_data = self.data[index * self.batch_size:(index + 1) * self.batch_size]
#         batch_labels = self.labels[index * self.batch_size:(index + 1) * self.batch_size]

#         return np.array(batch_data), np.array(batch_labels)

for comm_round in range(comms_round):

    # Get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()

    # Initial list to collect local model weights after scaling
    local_weight_list = []

    # Randomize client data - using keys
    client_names = list(clients_batched.keys())
    random.shuffle(client_names)

    for client in tqdm(client_names, desc='Progress Bar'):
        local_model = SimpleMLP.build()

        local_model.compile(
            loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy']
        )

        # Set local model weight to the weight of the global model
        local_model.set_weights(global_weights)
        
#         generator = DataGenerator(clients_batched[client][0], clients_batched[client][1], batch_size=15)

#         # Fit local model with client's data using generator
#         local_model.fit(
#             generator,
#             epochs=2,
#             verbose=2
#         )
        
        # Fit local model with client's data
        local_model.fit(
            np.array(clients_batched[client][0]),
            np.array(clients_batched[client][1]),
            epochs=2,
            batch_size= 8,
            verbose= 2
        )

        # Get the scaled model weights and add to the list
        weights = local_model.get_weights()
        local_weight_list.append(weights)

        # Clear the session to free memory after each communication round
        K.clear_session()

    # Calculate the average weights across all clients for each layer
    average_weights = avg_weights(local_weight_list)

    # Update the global model with the average weights
    global_model.set_weights(average_weights)
    
    # Optionally, you can also test the global model at this point using a separate test dataset
    global_acc, global_loss = test_model(x_test, y_test, global_model, comm_round)
    acc3.append(global_acc)


Progress Bar:   0%|                                                                              | 0/4 [00:00<?, ?it/s]

Epoch 1/2
6250/6250 - 491s - loss: 1.2722 - accuracy: 0.5506 - 491s/epoch - 79ms/step
Epoch 2/2
6250/6250 - 489s - loss: 1.0675 - accuracy: 0.6241 - 489s/epoch - 78ms/step


Progress Bar:  25%|█████████████████                                                   | 1/4 [16:45<50:17, 1005.93s/it]

Epoch 1/2
9375/9375 - 716s - loss: 1.2196 - accuracy: 0.5691 - 716s/epoch - 76ms/step
Epoch 2/2
9375/9375 - 700s - loss: 0.9981 - accuracy: 0.6473 - 700s/epoch - 75ms/step


Progress Bar:  50%|██████████████████████████████████                                  | 2/4 [40:51<42:09, 1264.83s/it]

...

313/313 [==============================] - 38s 119ms/step - loss: 1.2049 - accuracy: 0.5935
comm_round: 0 | global_acc: 59.350% | global_loss: 1.2049115896224976


Progress Bar:   0%|                                                                              | 0/4 [00:00<?, ?it/s]

Epoch 1/2
3125/3125 - 235s - loss: 1.0199 - accuracy: 0.6385 - 235s/epoch - 75ms/step
Epoch 2/2
3125/3125 - 232s - loss: 0.9246 - accuracy: 0.6722 - 232s/epoch - 74ms/step


Progress Bar:  25%|█████████████████▎                                                   | 1/4 [08:47<26:23, 527.76s/it]


MemoryError: Unable to allocate 2.29 GiB for an array with shape (200000, 32, 32, 3) and data type float32

In [ ]:
import matplotlib.pyplot as plt
plt.plot(acc3)

In [ ]:
acc3=np.array(acc3)

In [ ]:
global_model.evaluate(x_test,y_test)

In [ ]:
np.save("acc_fedavg_cifar.npy",acc3)

In [ ]:
global_model.save("fedavg_cifar.h5")

In [ ]:
a = np.load("acc_fedavg_cifar.npy")

In [ ]:
plt.plot(a)